#### Importing needed libraries and dependencies

In [1]:
import os 
import ee
import time
import rasterio
import math
import pyproj
import libpysal
import networkx as nx
import numpy as np
import osmnx as ox
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
import shapely.geometry
from rasterio.warp import reproject
from rasterstats import zonal_stats
from pylandtemp import single_window
from shapely.geometry import Point, Polygon, LineString

#### Setting initial varibles

Adjust the variables below in order to fine tune the calculations done throughout the code

In [4]:
#Set the directory for download and directory of counter file
os.chdir('/Users/winke/Documents/University/Thesis/Extracts')

#Read the counter data into the program
df = pd.read_csv("/Users/winke/Documents/University/Thesis/Extracts/weekly_new.csv")

#Checking working directory and creating path
directory = os.getcwd() #use working directory or desired location
connect = "/"

In [5]:
#Select cities and create a list that can be iterated through
citynames_df = df[['city', 'espg']]
citynames_df = citynames_df.drop_duplicates()
citynames_df

,city,espg
0,Amsterdam,32631
13,Rotterdam,32631
52,New York,32618
66,Seattle,32610
80,Utrecht,32631
101,"Norfolk, Virginia",32618
118,Arlington County,32618
143,Austin,32614
156,Minneapolis,32615
160,Almere,32631


#### Calculate ESPG values for CSV 
Calculate the ESPG for each city in order to have the correct projection when working with the data

In [ ]:
#Define the function to conver the coordinates into UTM coordinates
def convert_wgs_to_utm(lon, lat):
    utm_band = str((math.floor((lon + 180) / 6 ) % 60) + 1)
    if len(utm_band) == 1:
        utm_band = '0'+utm_band
    if lat >= 0:
        epsg_code = '326' + utm_band
    else:
        epsg_code = '327' + utm_band
    return epsg_code

#Loop through all coordinates and establish ESPG
espg_list = []
lat = df['latitude'].to_list()
lon = df['longitude'].to_list()

for coord in zip(lat, lon):
    espg = convert_wgs_to_utm(coord[1], coord[0])

    espg_list.append(espg)

#Add onto existing dataframe
df1 = df.copy()
df1['espg'] = espg_list

#### Creating city folders and extracting city bounds

In order to be able to extract geographic variables for each of the cities included in the analysis, the bounds of the city will have to be determined. The bounds are extracted from the OSMnx API and converted into a rectangular shape, ecompassing the total extent of the city, as defined by OSMnx. Additionally, a small buffer is added to ensure that counter locations at the edge of the city will be able to be included in the analysis.

In [ ]:
# ### Extracting city bound ###

for index, row in citynames_df.iterrows():
    
    city = row['city']
    path = directory + connect + city

    #Check if folder exists and if not, create it
    if not os.path.exists(path):
        os.makedirs(path)

    # download/model a street network for some city
    G = ox.graph_from_place(city, network_type="drive")

    #Save to geopackage
    ox.save_graph_geopackage(G, city + "/network.gpkg")

    #Save to graphml
    ox.io.save_graphml(G, city + "/network.graphml")
    
    # Retrieve only edges from the graph
    nodes_proj, edges = ox.graph_to_gdfs(G, nodes=True, edges=True)
    
    #Get the bounding box of all the edges, this will be the are of interest for each city
    bbox_env = edges.unary_union.envelope

    #Create buffer around city to make sure all count locations are captured
    bbox_env_buffer = bbox_env.buffer(0.08, cap_style=3, join_style=2)

    #Save bounding box as gpkg
    envgdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries(bbox_env_buffer))
    envgdf.to_file(city + connect + "citybound_gpkg.gpkg", driver="GPKG")
    
    print(city + " is finished")

    # Leave time in between extractions to save server connections
    time.sleep(20)

#### Download OSMNx point data

Part of the variables that are being generated are statistics of POI points sorrounding each counter location. To calculate these statistics, the locations of these POI points are downloaded and saved in a city specific folder. The POI location points are extracted from the OSMnx API and saved for each indiviual city in the corresponding folder. 

For each city, the city bound files are used in order to determine the extent to which these POI points are meant to be downloaded. Each categorical collection of POI points is downloaded and saved as a GeoPackage file. 

Dependencies:
1. City folders created
2. City bounds downloaded


In [ ]:
### Downloading point data ###
for index, row in citynames_df.iterrows():
    city = row['city']
    espg = row['espg']
    city_path = directory + connect + city

    #Read the city bound that has been downloaded
    citybound = gpd.read_file(city_path + '/citybound_gpkg.gpkg')

    #Create shapely polygon from citybounds
    for index, p in citybound.iterrows():
        poly = p.geometry
    
    #Extracting bus stops
    bus_stops = ox.geometries_from_polygon(poly, tags={'highway': 'bus_stop'})
    bus_stops = bus_stops.to_crs(espg)
    bus_stops = bus_stops[['geometry']]
    bus_stops.to_file(city_path + "/bus_stops.gpkg", driver="GPKG")

    time.sleep(10)

    #Extracting restaurants
    restaurants = ox.geometries_from_polygon(poly, tags={'amenity': ['bar', 'pub', 'restaurant', 'cafe']})
    restaurants = restaurants.to_crs(espg)
    restaurants = restaurants[['geometry']]
    restaurants.to_file(city_path + "/restaurants.gpkg", driver="GPKG")

    time.sleep(10)

    #Extracting bike POIs
    bikePOI = ox.geometries_from_polygon(poly, tags={'amenity': ['bicycle_parking', 'bicycle_repair_station', 'bicycle_rental']})
    bikePOI.to_crs(espg)
    bikePOI = bikePOI[['geometry']]
    bikePOI.to_file(city_path + "/bikePOI.gpkg", driver="GPKG")

    time.sleep(10)

    #Extracting daily shops
    shops = ox.geometries_from_polygon(poly, tags={'shop': ['department_store', 'supermarket', 'convenience']})
    shops.to_crs(espg)
    shops = shops[['geometry']]
    shops.to_file(city_path + "/daily_shops.gpkg", driver="GPKG")

    time.sleep(10)

    #Extracting business shops
    shops_2 = ox.geometries_from_polygon(poly, tags={'shop': ['clothes', 'jewelry', 'shoes', 'tailor', 'beauty', 'cosmetics', 'hairdresser',
                                                    'doityourself', 'garden_center', 'hardware', 'mall', 'department_store']})
    shops_2.to_crs(espg)
    shops_2 = shops_2[['geometry']]
    shops_2.to_file(city_path + "/business_shops.gpkg", driver="GPKG")

    # time.sleep(10)

    #Extracting traffic signals
    tfs = ox.geometries_from_polygon(poly, tags={'highway': 'traffic_signals'})
    tfs.to_crs(espg)
    tfs = tfs[['geometry']]
    tfs.to_file(city_path + "/traffic_signals.gpkg", driver="GPKG")

    time.sleep(10)

    #Extracting greenspace
    greenspace = ox.geometries_from_polygon(poly, tags={'leisure': ['garden', 'nature_reserve', 'park', 'pitch']})
    greenspace.to_crs(espg)
    greenspace_cut = greenspace[['geometry']]
    greenspace_cut.to_file(city_path + "/greenspace.gpkg", driver="GPKG")

    time.sleep(10)

    #Extracting eduPOI
    edu_POI = ox.geometries_from_polygon(poly, tags={'amenity': ['school', 'university', 'college']})
    edu_POI.to_crs(espg)
    edu_POI = edu_POI[['geometry']]
    edu_POI.to_file(city_path + "/eduPOI.gpkg", driver="GPKG")

    time.sleep(10)

    #Extracting trainstations
    station = ox.geometries_from_polygon(poly, tags={'railway': ['station']})
    station.to_crs(espg)
    station = station[['geometry']]
    station.to_file(city_path + "/t_stations.gpkg", driver="GPKG")

    time.sleep(10)

    #Extracting cyclelanes
    cycleways = ox.geometries_from_polygon(poly, tags={'cycleway': True, 'highway':'cycleway'})
    cycleways.to_crs(espg)
    cycleways_c = cycleways[['geometry']]

    #Create union out of all geometry extracted
    temp_list = []

    for index, x in cycleways_c.iterrows():
        temp_list.append(x.geometry)

    series = gpd.GeoSeries(temp_list)

    #Convert to gdf and export as gpkg
    cycleways_gdf = gpd.GeoDataFrame(geometry=gpd.GeoSeries(series))
    cycleways_gdf.to_file(city_path + "/cycleways.gpkg", driver="GPKG")
    
    print(city + " done")


#### Extracting Google Earth Engine raster data

The Google Earth Engine API is used in order to download global raster files for calculating additional geographical variables. The raster images are clipped down to the extent of each city using the city bounds. Once extracted, the images will have to be downloaded from Google Drive and inserted into each city folder in order to be able to be used for later calculation. 

IMPORTANT: Google Earth Engine is not able to download raster files to the device locally, it will therefore connect to the users Google Drive account and save the raster files there, which will have to then be manually downloaded and added into the folder corresponding to each of the cities.

Dependencies:
1. City bounds extracted

In [ ]:
#Authenticate Earth Engine through Google Account (this accounts Google Drive will be used to save the data)
ee.Authenticate()
ee.Initialize()

In [ ]:
### Raster file extraction from GEE ###

for index, row in citynames_df.iterrows():
    
    city = row['city']
    espg = row['espg']
    year = str(row['year'])
    city_path = directory + connect + city

    #Read the city bound that has been downloaded
    citybound = gpd.read_file(city_path + '/citybound_gpkg.gpkg')

    #Create shapely polygon from citybounds
    for index, p in citybound.iterrows():
        poly = p.geometry

    #Get citybound coordinates for extraction
    xx, yy = poly.exterior.coords.xy
    x = xx.tolist()
    y = yy.tolist()
    poly_coords = []

    num = 0

    for coord in x:
        temp = []
        temp.append(x[num])
        temp.append(y[num])
        poly_coords.append(temp)
        num += 1

    #Define region as ee.Geometry
    region = ee.Geometry.Polygon(poly_coords)

    # ##NDVI##
    # #Downloading Sentinal 2 data
    # sen_2 = ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED').filterDate('2021-01-01', '2021-12-31').filterBounds(region).filterMetadata('CLOUDY_PIXEL_PERCENTAGE','less_than', 5)

    # sen_2 = sen_2.mean()
    
    # #Calculating NDVI from image collection
    # def calculate_ndvi(image):
    #     ndvi = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    #     return image.addBands(ndvi)

    # ndvi = calculate_ndvi(sen_2).select('NDVI')

    # #Export to GoogleDrive
    # task = ee.batch.Export.image.toDrive(**{
    #     'image': ndvi,
    #     'description': 'NVDI',
    #     'folder': city,
    #     'scale': 10,
    #     'crs': 'EPSG:' + str(espg),
    #     'region': region.getInfo()['coordinates']
    #     })

    # task.start()

    # time.sleep(10)

    # ##DEM##
    # #Downloading Sentinal 2 data
    # srtm = ee.Image('USGS/SRTMGL1_003')
    # srtm_s = srtm.select('elevation')

    # #Export to GoogleDrive
    # task = ee.batch.Export.image.toDrive(**{
    #     'image': srtm_s,
    #     'description': 'DEM',
    #     'folder': city,
    #     'scale': 30,
    #     'crs': 'EPSG:' + str(espg),
    #     'region': region.getInfo()['coordinates']
    #     })

    # task.start()

    # time.sleep(10)

    ##LST##
    #For the LST calculation, 3 raster files are needed. They are downloaded seperately and then used together in calculation.
    #Might have to adjust cloud cover percentage for certain cities
    #Downloading data
    landsat_8 = ee.ImageCollection('LANDSAT/LC08/C02/T1').filterDate(year+'-01-01', year+'-12-31').filterBounds(region).filterMetadata('CLOUD_COVER','less_than', 30)

    #Select LST band from recording
    red = landsat_8.select('B4')
    nir = landsat_8.select('B5')
    thermal = landsat_8.select('B10')

    red_mean = red.mean()
    nir_mean = nir.mean()
    thermal_mean = thermal.mean()

    #Export red to GoogleDrive
    task = ee.batch.Export.image.toDrive(**{
        'image': red_mean,
        'description': 'LST_red',
        'folder': city,
        'scale': 30,
        'crs': 'EPSG:' + str(espg),
        'region': region.getInfo()['coordinates']
        })

    task.start()

    #Export nir to GoogleDrive
    task = ee.batch.Export.image.toDrive(**{
        'image': nir_mean,
        'description': 'LST_nir',
        'folder': city,
        'scale': 30,
        'crs': 'EPSG:' + str(espg),
        'region': region.getInfo()['coordinates']
        })

    task.start()

    #Export red to GoogleDrive
    task = ee.batch.Export.image.toDrive(**{
        'image': thermal_mean,
        'description': 'LST_thermal',
        'folder': city,
        'scale': 30,
        'crs': 'EPSG:' + str(espg),
        'region': region.getInfo()['coordinates']
        })

    task.start()

    # # time.sleep(10)

    # ##Land Cover##
    # #Downloading Sentinal 2 data
    # lc_2 = ee.ImageCollection('ESA/WorldCover/v100').filterDate('2020-01-01', '2020-12-31').filterBounds(region).first()

    # #Select LST band from recording
    # lc = lc_2.select('Map')

    # #Export to GoogleDrive
    # task = ee.batch.Export.image.toDrive(**{
    #     'image': lc,
    #     'description': 'landcover',
    #     'folder': city,
    #     'scale': 30,
    #     'crs': 'EPSG:' + str(espg),
    #     'region': region.getInfo()['coordinates']
    #     })

    # task.start()

    # time.sleep(10)

    # ##Pop grid##
    # #Downloading WorldPop Global data
    # worldpop = ee.ImageCollection('WorldPop/GP/100m/pop').filterDate('2020-01-01', '2020-12-31').filterBounds(region)
    
    # worldpop = worldpop.mean()
    # worldpop_2 = worldpop.select('population')

    # #Export to GoogleDrive
    # task = ee.batch.Export.image.toDrive(**{
    #     'image': worldpop_2,
    #     'description': 'population',
    #     'folder': city,
    #     'scale': 100,
    #     'crs': 'EPSG:' + str(espg),
    #     'region': region.getInfo()['coordinates']
    #     })

    # task.start()
    print('GEE done for ' + city)

#### Create buffers

In order to calculate the statistics of the various spatial variables extracted above, buffers are created around each counter location. These buffers are then saved as a geopackage file in each city folder in order to be extracted for calculating the values for the different spatial variables. 

In [ ]:
### Create buffers ###
#Read the counter data into the program

count = 0
l = []

for index, row in citynames_df.iterrows():
    city = row['city']
    espg = row['espg']
    city_path = directory + connect + city

    df2 = df[df['city'] == city]
    df3 = df2[['name', 'latitude', 'longitude']]

    #Convert to geopandas with point 
    points = [Point(xy) for xy in zip(df3.longitude, df3.latitude)]
    df_1 = pd.DataFrame({'geometry': points})
    gdf = gpd.GeoDataFrame(df_1, geometry='geometry')

    # set the lat/lon CRS
    gdf.crs = pyproj.CRS(4326)

    # convert the CRS to a projected CRS
    gdf = gdf.to_crs(pyproj.CRS(espg))

    # create the 500m buffer around each point
    buffered_geometries = [point.buffer(250) for point in gdf['geometry']]

    # create a new geodataframe with the buffer geometries
    buffer_gdf = gpd.GeoDataFrame(geometry=buffered_geometries)

    # set the projected CRS for the buffer geodataframe
    buffer_gdf.crs = pyproj.CRS(espg)

    buffer_gdf.to_file(city_path + "/buffer_gpkg/point_buffers1.gpkg", layer='buffers', driver="GPKG")

#### Extracting network statistics from OSMnx

Part of the variables that are used in counter prediction are network statistics that are based on the OSMnx network sorrounding each of the counter locations. The below code extracts the networks around each of the counter locations and calculates the basic network statistics provided by OSMnx.


In [ ]:
### Extracting network statistics ###

df_keys = []
df_values = []
df_index = []
locations = []
key_saved = False

for index, row in df.iterrows():
    city = row['city']
    lat = row['latitude']
    lon = row['longitude']
    point = (lat, lon)
    city_path = directory + connect + city
    
    #Download network graph for area
    G = ox.graph_from_point(point, dist=250, network_type="drive", truncate_by_edge=True)
    G_proj = ox.project_graph(G)

    #Calculate statistics around the particular point
    stats = ox.basic_stats(G)

    #Delete street per node proportion and calculate 3/4-way intersection cound
    del stats['streets_per_node_proportions']
    if 3 in stats['streets_per_node_counts'].keys():    
        in_3 = stats['streets_per_node_counts'][3]
    else:
        in_3 = 0
    if 4 in stats['streets_per_node_counts'].keys():
        in_4 = stats['streets_per_node_counts'][4]
    else:
        in_4 = 0
    
    stats['3_way_int_count'] = in_3
    stats['4_way_int_count'] = in_4

    #Extract all keys and values to make them into dataframe
    keys = list(stats.keys())
    values = list(stats.values())
    locations.append(row['name'])

    if key_saved == False:
        df_keys = keys
        key_saved = True

    df_values.append(values)
    df_index.append(index)
        
    (print('location', index, 'done'))

df1 = pd.DataFrame(df_values, columns=df_keys)
df1['locations'] = locations
df1

In [ ]:
df1 = pd.DataFrame(df_values, columns=df_keys)
df1['locations'] = locations
df1

In [ ]:
#Save results to a csv in directory
df1.to_csv("network_statistics_1.csv")

#### Caclculating median speed of sorrounding roads

In [ ]:
## Calculating the median speed of roads sorrounding the buffer
def convert_to_kmh(val):
    if "mph" in val:
        return float(val.split(" ")[0].strip("'")) * 1.60934
    else:
        return float(val.strip("' "))

median_speed_l = []

for index, row in citynames_df.iterrows():
    
    city = row['city']
    espg = row['espg']
    city_path = directory + connect + city

    #Load in cycling network data and buffer data
    drive_net = gpd.read_file(city_path + "/network_drive.gpkg", layer='edges').to_crs(espg)
    buffer_gdkg = gpd.read_file(city_path + "/buffer_gpkg/point_buffers1.gpkg").to_crs(espg)

    for index, buffer in buffer_gdkg.iterrows():
        clip = drive_net.clip(buffer.geometry)
        median_speed = clip['maxspeed'].values

        #Check for different conditions of extracted speed and convert to numpy median score
        b = []
        for i in median_speed:
            if i != '':
                if i[0] == '[':
                    values = i[1:-1].split(',')
                    b.extend([convert_to_kmh(val.strip(" ") ) for val in values])
                else:
                    b.append(convert_to_kmh(i))
        m_speed = np.median(np.array(b))
        median_speed_l.append(m_speed)

df8 = pd.DataFrame()
df8['median_speed'] = median_speed_l
df8

In [ ]:
pd.set_option('display.max_rows', 10)
df8

In [ ]:
df8.to_csv('median_speed.csv')

#### Calculating point variables

These functions use the POI points extracted from OSMnx API to calculate the amount that each POI occurs around each counter location. This calculation loads both the buffers and OSMnx point location to calculate the variables. 

Dependencies:
1. Buffers created
2. OSMnx point data downloaded

In [ ]:
### Calculate point variables ###

#Check for overlap between each polygon and points
bike_point_list = []
bus_stops_list = []
restaurant_list = []
dshops_list = []
bshops_list = []
traffic_s_list = []

for index, row in citynames_df.iterrows():
    
    city = row['city']
    espg = row['espg']
    city_path = directory + connect + city

    #Connect folder and load in points
    bike_shops = gpd.read_file(city_path + "/bikePOI.gpkg").to_crs(espg)
    bus_stops = gpd.read_file(city_path + "/bus_stops.gpkg").to_crs(espg)
    restaurants = gpd.read_file(city_path + "/restaurants.gpkg").to_crs(espg)
    # daily_shops = gpd.read_file(city_path + "/daily_shops.gpkg").to_crs(espg)
    business_shops = gpd.read_file(city_path + "/business_shops.gpkg").to_crs(espg)
    traffic_sig = gpd.read_file(city_path + "/traffic_signals.gpkg").to_crs(espg)

    #Load in buffers
    buffer_gdkg = gpd.read_file(city_path + "/buffer_gpkg/point_buffers1.gpkg").to_crs(espg)

    for index, row in buffer_gdkg.iterrows():
        temp = row.geometry
        bike_point_counts = 0
        bus_stop_counts = 0
        restaurant_counts = 0
        dshop_counts = 0
        bshop_counts = 0
        traffic_counts = 0

        #overlap bike shops
        for index, row in bike_shops.iterrows():
            temp1 = row.geometry
            
            if temp.contains(temp1):
                bike_point_counts += 1

        #overlap bus stops
        for index, row in bus_stops.iterrows():
            temp1 = row.geometry
            
            if temp.contains(temp1):
                bus_stop_counts += 1

        #overlap restaurants
        for index, row in restaurants.iterrows():
            temp1 = row.geometry
            
            if temp.contains(temp1):
                restaurant_counts += 1

        #overlap daily shops
        for index, row in daily_shops.iterrows():
            temp1 = row.geometry
            
            if temp.contains(temp1):
                dshop_counts += 1
        
        #overlap business shops
        for index, row in business_shops.iterrows():
            temp1 = row.geometry
            
            if temp.contains(temp1):
                bshop_counts += 1
        
        #overlap traffic signals
        for index, row in traffic_sig.iterrows():
            temp1 = row.geometry
            
            if temp.contains(temp1):
                traffic_counts += 1

        bike_point_list.append(bike_point_counts)
        bus_stops_list.append(bus_stop_counts)
        restaurant_list.append(restaurant_counts)
        dshops_list.append(dshop_counts)
        bshops_list.append(bshop_counts)
        traffic_s_list.append(traffic_counts)

df5 = pd.DataFrame()

df5['bike_points'] = bike_point_list
df5['bus_stops'] = bus_stops_list
df5['restaurants'] = restaurant_list
df5['daily_shops'] = dshops_list
df5['business_shops'] = bshops_list
df5['traffic_signals'] = traffic_s_list
 
df5

In [ ]:
#Export dataframe to csv
df5.to_csv('point_data_export_4.csv')

#### Calculating distance to POI variables

Using the greenspace and education POI locations, this algorithm calculates the network distance between each counter location and their closest greenspace and education POI. 

In [ ]:
### Calculating distance variabels to POIs ###

greenspace_dist_list = []
bike_dist_list = []
edu_dist_list = []
station_dist_list = []

for index, row in citynames_df.iterrows():
    
    city = row['city']
    espg = row['espg']
    city_path = directory + connect + city

    #Creating dataframe
    df2 = df[df['city'] == city]
    df3 = df2[['name', 'latitude', 'longitude']].copy()

    #Convert to geopandas with point 
    geometry = [Point(xy) for xy in zip(df3.longitude, df3.latitude)]
    df4 = df3.drop(['longitude', 'latitude'], axis=1)
    gdf = gpd.GeoDataFrame(df4, crs="EPSG:4326", geometry=geometry)
    gdf = gdf.to_crs(espg)

    #Load in street network graph
    G = ox.load_graphml(city_path + "/network.graphml")

    #Load in greenspace polygons
    greenspace = gpd.read_file(city_path + "/greenspace.gpkg").to_crs(espg)

    for index, point in gdf.iterrows():
        #Find closest polygon and get centroid
        polygon_index = greenspace.distance(point.geometry).sort_values().index[0]
        nearest_centroid = greenspace.loc[polygon_index].geometry.centroid

        #Extract coordinates of point
        x, y = point.geometry.coords.xy
        x = x[0]
        y = y[0]

        #Find closest node
        node_counter = ox.distance.nearest_nodes(G, x, y)
        
        #Extract coordinates of point
        xx, yy = nearest_centroid.xy
        xx = xx[0]
        yy = yy[0]

        #Find closest node
        node_greenspace = ox.distance.nearest_nodes(G, xx, yy)

        #Calculate shortest path
        try:
            nx.shortest_path_length(G, node_counter, node_greenspace)
        
        except:
            s_path = 0
        
        else:
            s_path = nx.shortest_path_length(G, node_counter, node_greenspace)
        
        greenspace_dist_list.append(s_path)

    #Load in greenspace polygons
    bikePOI = gpd.read_file(city_path + "/bikePOI.gpkg").to_crs(espg)

    #Import points as gdf and loop through them
    for index, point in gdf.iterrows():
        polygon_index = bikePOI.distance(point.geometry).sort_values().index[0]
        nearest_bike = bikePOI.loc[polygon_index].geometry.centroid

        #Extract coordinates of point
        x, y = point.geometry.coords.xy
        x = x[0]
        y = y[0]

        #Find closest node
        node_counter = ox.distance.nearest_nodes(G, x, y)
        
        #Extract coordinates of point
        xx, yy = nearest_bike.xy
        xx = xx[0]
        yy = yy[0]
        
        #Find closest node
        node_greenspace = ox.distance.nearest_nodes(G, xx, yy)

        #Calculate shortest path
        try:
            nx.shortest_path_length(G, node_counter, node_greenspace)
        
        except:
            s_path = 0
        
        else:
            s_path = nx.shortest_path_length(G, node_counter, node_greenspace)

        bike_dist_list.append(s_path)

    #Load in eduPOI polygons
    eduPOI = gpd.read_file(city_path + "/eduPOI.gpkg").to_crs(espg)

    #Import points as gdf and loop through them
    for index, point in gdf.iterrows():
        polygon_index = eduPOI.distance(point.geometry).sort_values().index[0]
        nearest_edu = eduPOI.loc[polygon_index].geometry.centroid

        #Extract coordinates of point
        x, y = point.geometry.coords.xy
        x = x[0]
        y = y[0]

        #Find closest node
        node_counter = ox.distance.nearest_nodes(G, x, y)
        
        #Extract coordinates of point
        xx, yy = nearest_edu.xy
        xx = xx[0]
        yy = yy[0]
        
        #Find closest node
        node_edu = ox.distance.nearest_nodes(G, xx, yy)

        #Calculate shortest path
        try:
            nx.shortest_path_length(G, node_counter, node_edu)
        
        except:
            s_path = 0
        
        else:
            s_path = nx.shortest_path_length(G, node_counter, node_edu)

        edu_dist_list.append(s_path)

    #Load in eduPOI polygons
    stations = gpd.read_file(city_path + "/t_stations.gpkg").to_crs(espg)

    #Import points as gdf and loop through them
    for index, point in gdf.iterrows():
        try:
            polygon_index = stations.distance(point.geometry).sort_values().index[0]
        except:
            s_path = 0
            station_dist_list.append(s_path)
            continue

        nearest_station = stations.loc[polygon_index].geometry.centroid

        #Extract coordinates of point
        x, y = point.geometry.coords.xy
        x = x[0]
        y = y[0]

        #Find closest node
        node_counter = ox.distance.nearest_nodes(G, x, y)
        
        #Extract coordinates of point
        xx, yy = nearest_station.xy
        xx = xx[0]
        yy = yy[0]
        
        #Find closest node
        node_greenspace = ox.distance.nearest_nodes(G, xx, yy)

        #Calculate shortest path
        try:
            nx.shortest_path_length(G, node_counter, node_greenspace)
        
        except:
            s_path = 0
        
        else:
            s_path = nx.shortest_path_length(G, node_counter, node_greenspace)

        station_dist_list.append(s_path)

    print(city + " is done")

df6 = pd.DataFrame()
df6['dist_to_greenspace'] = greenspace_dist_list
df6['dist_to_bikePOI'] = bike_dist_list
df6['dist_to_eduPOI'] = edu_dist_list
df6['dist_to_railstation'] = station_dist_list

df6

In [ ]:
#Export dataframe to csv
df6.to_csv('dist_to_green_bike.csv')

#### Calculate the total length of cyclepath in buffer

Using the downloaded cycling path network, this function calculates the total length of cycling infrastructure found in te buffer around each counter location. 

Dependencies:
1. Buffers created
2. Cycle infrastructure downloaded

In [ ]:
### Calculate total length of cyclepaths ###

total_cycle_list = []

for index, row in citynames_df.iterrows():
    
    city = row['city']
    espg = row['espg']
    city_path = directory + connect + city

    #Load in cycling network data and buffer data
    cycleways = gpd.read_file(city_path + "/cycleways.gpkg").to_crs(espg)
    buffer_gdkg = gpd.read_file(city_path + "/buffer_gpkg/point_buffers.gpkg").to_crs(espg)

    for index, buffer in buffer_gdkg.iterrows():
        clip = cycleways.clip(buffer.geometry)
        total_cycle = clip.length.sum()
        
        total_cycle_list.append(total_cycle)

df7  = pd.DataFrame()
df7['cycle_length'] = total_cycle_list

df7


In [ ]:
#Export dataframe to csv
df7.to_csv('cycle_length_2.csv')

#### Extract and calculate building area

This code extracts the buildings in the buffer around each counter location and calculates that total amount of area that the buildings cover within each buffer. 

In [ ]:
### Extract and calculate building area ###
#TODO: Make km2 instead of m2
build_dens_list = []
df_copy = df.copy()

#Extracting buildings and calculating building density around points
for index, row in citynames_df.iterrows():
    city = row['city']
    espg = row['espg']
    city_path = directory + connect + city
    
    #Load in buffers
    buffer_gdkg = gpd.read_file(city_path + "/buffer_gpkg/point_buffers.gpkg").to_crs(espg)
    df_cut = df_copy[df_copy['city'] == city]

    y=0
    for (index, row), (index1, row1) in zip(buffer_gdkg.iterrows(), df_cut.iterrows()):
        lat = row1['latitude']
        lon = row1['longitude']
        point = (lat, lon)

        #Extracting buildings to calculate building density and clip it to buffer
        buildings = ox.geometries_from_point(point, tags={'building': True}, dist=250)
        buildings = buildings.to_crs(espg)
        buffer = buffer_gdkg.iloc[[y]]
        y+=1
        c_buildings =buildings.clip(buffer)

        temp_l = []
        
        #calculate the building within clipped buffer
        for index, x in c_buildings.iterrows():
            temp_l.append(x.geometry)

        b_series = gpd.GeoSeries(temp_l).area
        b_total_area = b_series.sum()
        b_total_area = b_total_area / 1000000
        build_dens_list.append(b_total_area)

    print(city + ' done')
    break

df8 = pd.DataFrame()
df8['build_area'] = build_dens_list
df8

In [ ]:
#Export dataframe to csv
df8.to_csv('building.csv')

#### Calculate LST and save as new TIF file

This code takes the extracted Landsat 8 raster files in order to calculate a new raster file containing the Land Surface Temperature for each city. 

In [ ]:
### Calculate LST layer###

for index, row in citynames_df.iterrows():
    
    city = row['city']
    espg = row['espg']
    city_path = directory + connect + city

    #Load in each band that was exported from GEE
    red = rasterio.open(city_path + "/LST_red.tif")
    nir = rasterio.open(city_path + "/LST_nir.tif")
    thermal = rasterio.open(city_path + "/LST_thermal.tif")

    #Read the bands to be used for calculation
    redImage = red.read(1).astype('f4')
    nirImage = nir.read(1).astype('f4')
    thermalImage = thermal.read(1).astype('f4')

    lst_image_single_window = single_window(thermalImage, redImage, nirImage, unit='celcius')
    lst_image_single_window = lst_image_single_window - 273.15

    #Define affine transformation
    affine = red.transform
    crs_red = red.crs

    #Create new raster file with calculated variables
    with rasterio.open(
        city_path + "/LST_calculated.tif",
        mode="w",
        driver="GTiff",
        height=lst_image_single_window.shape[0],
        width=lst_image_single_window.shape[1],
        count=1,
        dtype=lst_image_single_window.dtype,
        crs=crs_red,
        transform=affine,
                        ) as new_dataset:
            new_dataset.write(lst_image_single_window, 1)


#### Calculate raster statistics

In [7]:
### Calculate raster statistics ###

#Define lists for each variable
ndvi_mean_lst = []
ndvi_std_lst = []
dem_mean_lst = []
dem_std_lst = []
lst_mean_lst = []
lst_std_lst = []
lc_entropy_lst = []
pop_sum_lst = []

def entropy(x):
    unique = np.unique(x, return_counts=True)
    unique_count = len(unique[0])
    unique_size = unique[1]
    total_size = x.size
    
    total = 0.0

    for y in unique_size:
        r = (y/total_size)*np.log(y/total_size)
        total = r + total

    c_entropy = (total/np.log(unique_count))*-1

    return c_entropy

for index, row in citynames_df.iterrows():
    city = row['city']
    espg = row['espg']
    city_path = directory + connect + city

    # #Load in the buffers around the counter locations
    # buffer_gdkg = gpd.read_file(city_path + "/buffer_gpkg/point_buffers.gpkg").to_crs(espg)

    # ##NDVI##
    # #Load in population file
    # ndvi = rasterio.open(city_path + "/NVDI.tif")
    # arr = ndvi.read(1)
    # affine = ndvi.transform

    # statistics = zonal_stats(buffer_gdkg, arr, affine=affine, stats=['mean', 'std'])

    # for x in statistics:
    #     ndvi_mean = x['mean']
    #     ndvi_std = x['std']

    #     ndvi_mean_lst.append(ndvi_mean)
    #     ndvi_std_lst.append(ndvi_std)

    # ##DEM##
    # #Load in population file
    # dem = rasterio.open(city_path + "/DEM.tif")
    # arr = dem.read(1)
    # affine = dem.transform

    # statistics = zonal_stats(buffer_gdkg, arr, affine=affine, stats=['mean', 'std'])

    # for x in statistics:
    #     dem_mean = x['mean']
    #     dem_std = x['std']

    #     dem_mean_lst.append(dem_mean)
    #     dem_std_lst.append(dem_std)


    ##LST##
    #Load in lst files
    lst = rasterio.open(city_path + "/LST_calculated.tif")
    arr = lst.read(1)
    affine = lst.transform

    # #Load in buffer file
    buffer_gdkg = gpd.read_file(city_path + "/buffer_gpkg/point_buffers1.gpkg").to_crs(espg)

    statistics = zonal_stats(buffer_gdkg, arr, affine=affine, stats=['mean', 'std'])

    for x in statistics:
        lst_mean = x['mean']
        lst_std = x['std']

        lst_mean_lst.append(lst_mean)
        lst_std_lst.append(lst_std)

    
    # ##Populationn density##
    # #Load in population files
    # pop = rasterio.open(city_path + "/population.tif")
    # arr = pop.read(1)
    # affine = pop.transform

    # # #Load in buffer file
    # buffer_gdkg = gpd.read_file(city_path + "/buffer_gpkg/point_buffers.gpkg").to_crs(espg)

    # statistics = zonal_stats(buffer_gdkg, arr, affine=affine, stats=['sum'])

    # for x in statistics:
    #     pop_sum = x['sum']

    #     pop_sum_lst.append(pop_sum)

    
    # ##Land Cover entropy##
    # lc = rasterio.open(city_path + "/landcover.tif")
    # arr = lc.read(1)
    # affine = lc.transform

    # # #Load in buffer file
    # buffer_gdkg = gpd.read_file(city_path + "/buffer_gpkg/point_buffers.gpkg").to_crs(espg)

    # statistics = zonal_stats(buffer_gdkg, arr, affine=affine, stats=['majority'], add_stats={'entropy':entropy})

    # for x in statistics:
    #     lc_entropy = x['entropy']
    #     lc_entropy_lst.append(lc_entropy)

#Add all variables to dataframe
df9 = pd.DataFrame()
# df9['ndvi_mean'] = ndvi_mean_lst
# df9['ndvi_std'] = ndvi_std_lst
# df9['dem_mean'] = dem_mean_lst
# df9['dem_std'] = dem_std_lst
df9['lst_mean'] = lst_mean_lst
df9['lst_std'] = lst_std_lst
# df9['pop_sum'] = pop_sum_lst
# df9['lc_entropy'] = lc_entropy_lst

df9

/Users/winke/opt/anaconda3/envs/ox/lib/python3.11/site-packages/rasterstats/main.py:156: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if 'Point' in geom.type:
/Users/winke/opt/anaconda3/envs/ox/lib/python3.11/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/winke/opt/anaconda3/envs/ox/lib/python3.11/site-packages/rasterstats/main.py:156: ShapelyDeprecationWarning: The 'type' attribute is deprecated, and will be removed in the future. You can use the 'geom_type' attribute instead.
  if 'Point' in geom.type:
/Users/winke/opt/anaconda3/envs/ox/lib/python3.11/site-packages/rasterstats/io.py:313: UserWarning: Setting nodata to -999; specify nodata explicitly
  warnings.warn("Setting nodata to -999; specify nodata explicitly")
/Users/winke/opt/anaconda3/envs/ox/lib/p

,lst_mean,lst_std
0,15.439598,0.668205
1,17.103036,0.746097
2,18.937200,1.111123
3,17.850033,1.267783
4,16.099557,0.542547
...,...,...
907,19.641176,0.804630
908,15.029029,0.776854
909,20.430821,0.937948
910,19.271217,0.977173


In [8]:
#Export dataframe to csv
df9.to_csv('raster_calculations_4.csv')

#### Calculate spatial lag variable

In [ ]:
# Load the cycling counter data into a Pandas DataFrame
data = pd.read_csv("weekly_new.csv")

# Create a Block Weights spatial weight matrix based on country and city
weights = libpysal.weights.block_weights(data[['country', 'city']].values)

# Calculate the spatial lag variable using the weight matrix
spatial_lag = np.zeros(len(data))
for i in range(len(data)):
    neighbors = weights.neighbors[i]
    for neighbor in neighbors:
        spatial_lag[i] += data.iloc[neighbor]['counts_week']

# Add the spatial lag variable to the DataFrame
df9 = pd.DataFrame()
df9['spatial_lag'] = spatial_lag
df9

In [ ]:
df9.to_csv('spatial_lag.csv')